In [66]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('..')

In [63]:
import s3fs
from training_classes.nomenclature import Nomenclature
from data_import.bdd import PostGre_SQL_DB

In [8]:
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'http://minio.stable.innovation.insee.eu'})
bdd = PostGre_SQL_DB(fs=fs)
naf = Nomenclature(bdd, 'NAF2_1')

In [7]:
fs.ls("s3://groupe-1033/nomenclatures/")

['groupe-1033/nomenclatures/NAF',
 'groupe-1033/nomenclatures/PCS',
 'groupe-1033/nomenclatures/fasttext',
 'groupe-1033/nomenclatures/published_models']

In [15]:
fs.put('trainings/PCS/2020-11-23_5/config.yaml',
       "s3://groupe-1033/nomenclatures/PCS/2020-11-23/5/config.yaml")

In [104]:
save_dir = "trainings/NAF/2020-10-26_4"
best_weights = "best_model_17-0.01509"
results_file = 'test_results.csv'
results = pd.read_csv(os.path.join(save_dir, results_file),
                     converters={
                         'top_k_codes': lambda x: x[1:-1].replace("'", "").replace(' ', '').split(','),
                         'top_k_similarities': lambda s: [float(x.strip(' []')) for x in s.split(',')]})

In [105]:
results.iloc[0]

Unnamed: 0                                                            0
cabbi                                                        7300033530
gt                                                                8710A
gt_score                                                       0.916052
top_k_codes           [8710A, 8730A, 8730, 873, 871, 8710, 872, 8790...
top_k_similarities    [0.916052146652651, 0.7626523887121395, 0.6255...
Name: 0, dtype: object

In [58]:
naf.get_node('8810A').desc

'Aide à domicile'

In [57]:
naf.get_node('9700Z').desc

"Activités des ménages en tant qu''employeurs de personnel domestique"

In [56]:
bdd.read_from_sql("SELECT * FROM rp_final_2017 WHERE cabbi='7300050429';")[['rs_x', 'actet_x', 'profs_x', 'profi_x', 'profa_x']]

,rs_x,actet_x,profs_x,profi_x,profa_x
0,CROIX ROUGE FRANCAISE,SERVICE D AIDE ET D ACCOMPAGNEMENT A DOMICILE,AUXILIAIRE DE VIE SOCIALE DIPLOMEE,None,None


In [109]:
# stats on diff between 1 and 2 when gt is 1
results['top_1'] = results['gt'] == results['top_k_codes'].map(lambda x: x[0])
results['top_2_diff'] = results['top_k_similarities'].map(lambda x: x[0] - x[1])
results[results.top_1]['top_2_diff'].mean()

# Exploration Comparaison PCS

In [90]:
from data_import.csv_to_postgresql import read_table_description_file, import_csv_to_postegrsql
from training_utils import load_config, get_best_savedmodel
from run_top_k import load_model_from_save_dir, get_top_k, get_projection
from training_classes.similarity_model import SimilarityModel

In [91]:
config = load_config(save_dir)

In [92]:
save_dir = 'trainings/PCS/2020-11-23_5'
best_weights = get_best_savedmodel(save_dir)
nomenclature, data_cleaner, data_formatter, model, top1_classifier = load_model_from_save_dir(save_dir, best_weights)

Model: "similarity_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 42)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 42)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 42)]         0                                            
__________________________________________________________________________________________________
embedding (TokenFieldAndPositio (None, 42, 180)      42567840    input_1[0][0]                    
                                                                 input_2[0][0]     

In [5]:
results = pd.read_csv('different_preds_rp_final_2018.csv', delimiter=';')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
results.columns

Index(['cabbi', 'rs_x', 'actet_x', 'profs_x', 'profi_x', 'profa_x', 'index',
       'actet_c_m', 'i_mca_c', 'i_actet_c', 'profs_c', 'profs_c_c',
       'profs_c_m', 'i_profs_c', 'profi_c', 'profi_c_c', 'profi_c_m',
       'i_profi_c', 'profa_c', 'profa_c_c', 'profa_c_m', 'i_profa_c',
       'pcs_proj', 'pcs_code_0', 'pcs_score_0', 'pcs_code_1', 'pcs_score_1',
       'pcs_code_2', 'pcs_score_2', 'pcs_code_3', 'pcs_score_3', 'pcs_code_4',
       'pcs_score_4', 'pcs_code_5', 'pcs_score_5', 'pcs_code_6', 'pcs_score_6',
       'pcs_code_7', 'pcs_score_7', 'pcs_code_8', 'pcs_score_8', 'pcs_code_9',
       'pcs_score_9', 'prof_c', 'prof_c_c', 'prof_c_m', 'i_prof_c',
       'auto_coded_batch', 'auto_coded_recap', 'previous_auto_coded',
       'previous_auto_not_coded', 'previous_manual_coded',
       'previous_not_coded', 'gt_rang', 'codage_auto', 'encoded_correctly',
       'encoded_incorrectly', 'top1', 'top5', 'top10', 'not_found',
       'not_calculated', 'previous_auto_encoded_and_encoded

In [9]:
pcs = Nomenclature(bdd, 'PCS1')

In [10]:
rang_2 = results[results.gt_rang == 1][['rs_x', 'actet_x', 'profs_x', 'profi_x', 'profa_x', 'prof_c', 'pcs_code_0', 'pcs_code_1']][:1000]

In [11]:
rang_2['pcs_0'] = rang_2['pcs_code_0'].map(lambda x: pcs.get_node(x).desc)
rang_2['pcs_1'] = rang_2['pcs_code_1'].map(lambda x: pcs.get_node(x).desc)

In [12]:
rang_2

,rs_x,actet_x,profs_x,profi_x,profa_x,prof_c,pcs_code_0,pcs_code_1,pcs_0,pcs_1
1,CENTRE HOSPITALIER DE VERDUN,HOPITAL,AGENT TECHNIQUE,NaN,NaN,632K,621F,632K,Ouvriers qualifiés des travaux publics (salari...,Ouvriers qualifiés d''entretien général des bâ...
2,VILLE DE LYON,GROUPE SCOLAIRE,RESPONSABLE DE RESTAURANT SCOLAIRE,NaN,NaN,468A,488B,468A,Maîtrise de restauration : gestion d''établis...,Maîtrise de restauration : salle et service
4,MAIRIE AMIENS,ANIMATRICE,ANIMATRICE,NaN,NaN,422E,435B,422E,Animateurs socioculturels et de loisirs,Surveillants et aides-éducateurs des établisse...
6,UNIVERSITE DE CAEN NORMANDIE,ENSEIGNEMENT,TECHNICIENNE ADMINISTRATIVE,NaN,NaN,451E,523A,451E,Adjoints administratifs de la fonction publiqu...,Autres personnels administratifs de catégorie ...
8,ADECCO,PAYSAGE,PAYSAGISTE,NaN,NaN,631A,211J,631A,"Entrepreneurs en parcs et jardins, paysagistes",Jardiniers
...,...,...,...,...,...,...,...,...,...,...
3590,COUREGES,MODE,COMPTABILITEE,NaN,NaN,373C,461D,373C,Maîtrise et techniciens des services financier...,Cadres des services financiers ou comptables d...
3592,VEOLIA,EQUIPIER DE COLLECTE,EQUIPIER DE COLLECTE,NaN,NaN,684B,628E,684B,Ouvriers qualifiés de l''assainissement et du ...,Ouvriers non qualifiés de l''assainissement et...
3597,VILLE DE PARIS,CRECHE,EDUCATRICE JEUNES ENFANTS,NaN,NaN,434G,434D,434G,Educateurs spécialisés,Educateurs de jeunes enfants
3604,EUROPORTE,FERROVIAIRE,CONDUCTEUR DE TRAIN,NaN,NaN,654A,641B,654A,Conducteurs de véhicule routier de transport e...,Conducteurs qualifiés d''engins de transport g...


In [13]:
results.iloc[3597][['pcs_score_0', 'pcs_score_1']]

pcs_score_0     0.93659
pcs_score_1    0.875668
Name: 3597, dtype: object

In [93]:
input_columns = [c.replace('_repr', '_x') for c in config['trainings']['data']['input_columns']]
gt_column = config['trainings']['data']['gt_column']
alpha_tree_mod = config['post_process']['alpha_tree_mod']
beta_str_sim_mod = config['post_process']['beta_str_sim_mod']
nb_top_values = 10

df = pd.DataFrame(results.loc[3597]).T.reset_index()
del df['index']

cleaned_data, proj = get_projection(df, input_columns, 
                                    data_cleaner, data_formatter, model)
nom = data_formatter.nomenclature_distance.nomenclature
similarity_func = SimilarityModel.similarity_func
cleaned_main_field = cleaned_data[input_columns[0]].values.tolist()

base_similarities = similarity_func(np.expand_dims(proj[0], axis=0), nom.projections).numpy()[0]
similarities = base_similarities.copy()
if alpha_tree_mod:
    sim_alpha = nom.get_tree_score_modifiers(data_formatter.nomenclature_distance, base_similarities)
    similarities += alpha_tree_mod * nom.get_tree_score_modifiers(data_formatter.nomenclature_distance, base_similarities)
if beta_str_sim_mod:
    sim_beta = nom.get_trigram_dist_modifiers(cleaned_main_field[i], base_similarities)
    similarities += beta_str_sim_mod * nom.get_trigram_dist_modifiers(cleaned_main_field[i], base_similarities)
top_idx = (-similarities).argsort()[:nb_top_values]
top_codes = [nom.nodes[nom.projections_reverse_ind[i]].id for i in top_idx]
top_similarities = [similarities[i] for i in top_idx]

In [82]:
nom_projection_ind = {v: k for k, v in nom.projections_reverse_ind.items()}

In [94]:
nom_projection_ind['434D']

277

In [95]:
sim_beta[277]

0.18576279977432889

In [85]:
nom_projection_ind['434G']

280

In [96]:
sim_beta[280]

0.6218430130838938

In [87]:
cleaned_main_field

['crech']

In [88]:
cleaned_data

,index,level_0,cabbi,rs_x,actet_x,actet_c_m,i_mca_c,i_actet_c,profs_c_c,profs_c_m,...,previous_manual_encoded_and_encoded_correctly,previous_manual_encoded_and_encoded_incorrectly,previous_manual_encoded_and_not_encoded,previous_manual_encoded_and_top1,previous_manual_encoded_and_top5,previous_manual_encoded_and_top10,previous_manual_encoded_and_notfound,previous_not_encoded_and_encoded,previous_not_encoded_and_not_calculated,prof_x
0,0,3597,8353037002,vill paris,crech,NaN,C,CAM,434G,NaN,...,False,False,False,False,False,False,False,False,False,educ jeun enfant


In [89]:
input_columns

['actet_x', 'rs_x', 'prof_x']

In [98]:
top_codes, top_similarities

(['434G',
  '434D',
  '451F',
  '523A',
  '52',
  '524A',
  '522A',
  '351A',
  '333F',
  '425A'],
 [0.811912697363536,
  0.7459271707458557,
  0.6170084428208236,
  0.5360121720413512,
  0.5156312600347912,
  0.42806181155903417,
  0.363851893458688,
  0.28212801788727565,
  0.2788423211567317,
  0.2775545935330373])